In [3]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': 'data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = 'data/custom_data/Preprocesed Data/classes.txt'
path_to_data_yaml = 'data/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')

file_path = Path("data/data.yaml")

with open(file_path, "r", encoding="utf-8") as f:
    print(f.read())

Created config file at data/data.yaml

File contents:

path: data
train: train/images
val: validation/images
nc: 11
names:
- 01_Good
- 10_Overhang_Sag
- 11_Shifted_Layer
- 12_Warping
- 13_Delamination
- 14_Unclear
- 15_Not_Printing
- 4_Foreign_Body
- 6_Stringing
- 7_Spaghetti
- 9_Poor_Bridging



In [8]:
yaml_text = """path: data
train: train/images
val: val/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing
"""

Path("data/data.yaml").write_text(yaml_text, encoding="utf-8")
print(Path("data/data.yaml").read_text(encoding="utf-8"))

path: data
train: train/images
val: val/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing



In [6]:
from pathlib import Path
from collections import Counter

root = Path("data")
label_files = list(root.rglob("labels/*.txt")) + list(root.rglob("**/labels/**/*.txt"))
print("Found label files:", len(label_files))

counts = Counter()
max_id = -1

for f in label_files:
    for line in f.read_text(encoding="utf-8", errors="ignore").splitlines():
        if not line.strip():
            continue
        cls = int(float(line.split()[0]))
        counts[cls] += 1
        max_id = max(max_id, cls)

print("Class IDs present:", sorted(counts.keys()))
print("Max class ID:", max_id)
print("Counts:", dict(sorted(counts.items())))

Found label files: 6272
Class IDs present: [1, 2, 3, 4, 5, 6, 7, 8]
Max class ID: 8
Counts: {1: 2070, 2: 1118, 3: 1214, 4: 836, 5: 2492, 6: 1064, 7: 1326, 8: 3126}


In [7]:
import sys, torch
import ultralytics

print("Python exe:", sys.executable)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("Ultralytics version:", ultralytics.__version__)

Python exe: c:\Program Files\Python312\python.exe
Torch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 4060 Laptop GPU
Ultralytics version: 8.3.229


In [ ]:
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt" 
DEVICE = 0                        
PROJECT = "runs/gridsearch"       
GRID_EPOCHS = 5
FIXED_IMGSZ = 640                        
SEED = 42

Hyperparameter tuning with Grid-Search

In [2]:
from ultralytics import YOLO
from pathlib import Path
import itertools
import pandas as pd
import time

# CONFIG
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt"          
DEVICE = 0                        
PROJECT = "runs/gridsearch"       
GRID_EPOCHS = 5
FIXED_IMGSZ = 640                        
SEED = 42

# GRID SETUP
grid = {
    "lr0": [0.01, 0.001, 0.0001],
    "weight_decay": [0.0005, 0.005, 0.05],
    "box": [5, 7.5, 10],
    "cls": [0.25, 0.5],
    "freeze": [18, 19, 20]
}

Path(PROJECT).mkdir(parents=True, exist_ok=True)

keys = list(grid.keys())
combos = list(itertools.product(*[grid[k] for k in keys]))
print(f"Total runs: {len(combos)}")

rows = []
t0 = time.time()

for i, values in enumerate(combos, start=1):
    params = dict(zip(keys, values))
    run_name = "gs_" + "_".join([f"{k}={v}" for k, v in params.items()])

    print(f"\n[{i}/{len(combos)}] Training: {run_name}")
    model = YOLO(BASE_MODEL)

    train_results = model.train(
        data=DATA_YAML,
        epochs=GRID_EPOCHS,
        imgsz=FIXED_IMGSZ,
        lr0=params["lr0"],
        weight_decay=params["weight_decay"],
        optimizer="AdamW",
        box=params["box"],
        freeze=params["freeze"],
        batch=16,
        device=DEVICE,
        seed=SEED,
        deterministic=True,
        project=PROJECT,
        name=run_name,
        plots=False,           # faster
        save=True,
        val=True,
        patience=4            # early stopping helps speed grid search
    )

# After training, evaluate the best checkpoint on val to get a clean final score
    best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
    best_model = YOLO(str(best_path))
    val_metrics = best_model.val(data=DATA_YAML, split="val", device=DEVICE, plots=False)

    # Extract metrics (these attribute names are stable in Ultralytics v8.x)
    row = {
        "run_name": run_name,
        "save_dir": str(model.trainer.save_dir),
        "best_pt": str(best_path),
        **params,
        "imgsz": params.get("imgsz", FIXED_IMGSZ),
        "val_mAP50_95": float(val_metrics.box.map),     # mAP@[.5:.95]
        "val_mAP50": float(val_metrics.box.map50),      # mAP@.5
        "val_precision": float(val_metrics.box.mp),     # mean precision
        "val_recall": float(val_metrics.box.mr),        # mean recall
    }
    rows.append(row)

 # Write partial results continuously (so you don’t lose progress)
    df = pd.DataFrame(rows).sort_values("val_mAP50_95", ascending=False)
    df.to_csv(Path(PROJECT) / "grid_results.csv", index=False)

print(f"\nDone in {(time.time()-t0)/60:.1f} minutes.")
df.head(10)

Total runs: 162

[1/162] Training: gs_lr0=0.01_weight_decay=0.0005_box=5_cls=0.25_freeze=18
New https://pypi.org/project/ultralytics/8.4.7 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=18, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt

KeyboardInterrupt: 

In [6]:
from ultralytics import YOLO
from pathlib import Path
import itertools
import pandas as pd
import time

# CONFIG
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt"          
DEVICE = 0                        
PROJECT = "runs/final_gridsearch"       
GRID_EPOCHS = 15
FIXED_IMGSZ = 640                        
SEED = 42

# GRID SETUP
grid = {
    "lr0": [0.001],
    "weight_decay": [0.05],
    "box": [10],
    "cls": [0.25],
    "freeze": [18, 17, 16, 15, 14, 13]
}

Path(PROJECT).mkdir(parents=True, exist_ok=True)

keys = list(grid.keys())
combos = list(itertools.product(*[grid[k] for k in keys]))
print(f"Total runs: {len(combos)}")

rows = []
t0 = time.time()

for i, values in enumerate(combos, start=1):
    params = dict(zip(keys, values))
    run_name = "gs_" + "_".join([f"{k}={v}" for k, v in params.items()])

    print(f"\n[{i}/{len(combos)}] Training: {run_name}")
    model = YOLO(BASE_MODEL)

    train_results = model.train(
        data=DATA_YAML,
        epochs=GRID_EPOCHS,
        imgsz=FIXED_IMGSZ,
        lr0=params["lr0"],
        weight_decay=params["weight_decay"],
        optimizer="AdamW",
        box=params["box"],
        freeze=params["freeze"],
        batch=16,
        device=DEVICE,
        seed=SEED,
        deterministic=True,
        project=PROJECT,
        name=run_name,
        plots=False,           # faster
        save=True,
        val=True,
        patience=4            # early stopping helps speed grid search
    )

# After training, evaluate the best checkpoint on val to get a clean final score
    best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
    best_model = YOLO(str(best_path))
    val_metrics = best_model.val(data=DATA_YAML, split="val", device=DEVICE, plots=False)

    # Extract metrics (these attribute names are stable in Ultralytics v8.x)
    row = {
        "run_name": run_name,
        "save_dir": str(model.trainer.save_dir),
        "best_pt": str(best_path),
        **params,
        "imgsz": params.get("imgsz", FIXED_IMGSZ),
        "val_mAP50_95": float(val_metrics.box.map),     # mAP@[.5:.95]
        "val_mAP50": float(val_metrics.box.map50),      # mAP@.5
        "val_precision": float(val_metrics.box.mp),     # mean precision
        "val_recall": float(val_metrics.box.mr),        # mean recall
    }
    rows.append(row)

 # Write partial results continuously (so you don’t lose progress)
    df = pd.DataFrame(rows).sort_values("val_mAP50_95", ascending=False)
    df.to_csv(Path(PROJECT) / "grid_results.csv", index=False)

print(f"\nDone in {(time.time()-t0)/60:.1f} minutes.")
df.head(10)

Total runs: 6

[1/6] Training: gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25_freeze=18
New https://pypi.org/project/ultralytics/8.4.7 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=10, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=18, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt,

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000223275A4D60>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/15      1.19G      3.057      2.058      1.726         13        640: 100% ━━━━━━━━━━━━ 82/82 8.7it/s 9.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 6.7it/s 2.4s0.2s
                   all        488       1030      0.419      0.365       0.38       0.17

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/15      1.33G      2.876      1.858      1.681         13        640: 100% ━━━━━━━━━━━━ 82/82 8.7it/s 9.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 6.6it/s 2.4s0.2s
                   all        488       1030      0.426       0.51      0.453        0.2

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       4/15      1.33G      2.781      1.778      1.612          7        640: 100%

,run_name,save_dir,best_pt,lr0,weight_decay,box,cls,freeze,imgsz,val_mAP50_95,val_mAP50,val_precision,val_recall
3,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,15,640,0.508025,0.861306,0.811927,0.816516
4,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,14,640,0.508025,0.861306,0.811927,0.816516
5,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,13,640,0.508025,0.861306,0.811927,0.816516
2,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,16,640,0.429247,0.767557,0.746534,0.692702
0,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,18,640,0.424083,0.766408,0.760759,0.700164
1,gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.05,10,0.25,17,640,0.424083,0.766408,0.760759,0.700164


In [13]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path

PROJECT = "runs/final_gridsearch"

df = pd.read_csv(Path(PROJECT) / "grid_results.csv").sort_values("val_mAP50_95", ascending=False)
best = df.iloc[0].to_dict()
print("Best config:", best)

final_name = "train_final"
model = YOLO(BASE_MODEL)
model.train(data="data/data.yaml",
            epochs=200,
            imgsz=FIXED_IMGSZ,
            lr0=float(best["lr0"]),
            weight_decay=float(best["weight_decay"]),
            optimizer="AdamW",
            box=float(best["box"]),
            freeze=int(best["freeze"]),
            cls=float(best["cls"]),
            batch=16,
            device=DEVICE,
            seed=SEED,
            deterministic=True,
            project="runs/detect",
            name=final_name,
            plots=True,
            val=True,
            patience=100,
            translate=0.2           
)

best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
last_path = Path(model.trainer.save_dir) / "weights" / "last.pt"
print("✅ Best checkpoint:", best_path)
print("✅ Last checkpoint:", last_path)

Best config: {'run_name': 'gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25_freeze=15', 'save_dir': 'C:\\Users\\User\\Documents\\GitHub\\Real-Time-Defect-Detection-Capstone-Project\\runs\\final_gridsearch\\gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25_freeze=15', 'best_pt': 'C:\\Users\\User\\Documents\\GitHub\\Real-Time-Defect-Detection-Capstone-Project\\runs\\final_gridsearch\\gs_lr0=0.001_weight_decay=0.05_box=10_cls=0.25_freeze=15\\weights\\best.pt', 'lr0': 0.001, 'weight_decay': 0.05, 'box': 10, 'cls': 0.25, 'freeze': 15, 'imgsz': 640, 'val_mAP50_95': 0.508025219553579, 'val_mAP50': 0.8613060163618782, 'val_precision': 0.811927262850669, 'val_recall': 0.816516220968015}
New https://pypi.org/project/ultralytics/8.4.7 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=10.

In [14]:
model = YOLO("runs/detect/train_final/weights/best.pt")

In [15]:
model.predict(source="data/val/images",
              save=True,
              name="val"
)

print("✅ Saved validation predictions under runs/detect/predict*")


image 1/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\val\images\00624767-right_77photo_2024-10-04T11_08_33-787900.jpg: 416x640 1 7_Spaghetti, 58.8ms
image 2/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\val\images\00b9f800-left_163photo_2024-10-02T14_05_41-353505.jpg: 416x640 1 6_Stringing, 44.2ms
image 3/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\val\images\0116188c-left_67photo_2024-10-02T13_11_10-9292.jpg: 416x640 1 6_Stringing, 15.3ms
image 4/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\val\images\012f441e-left_photo_57_2024-10-22_16-54-09.jpg: 416x640 1 13_Delamination, 7.1ms
image 5/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\val\images\01491492-157photo_2024-08-21T16_29_57-15868.jpg: 416x640 2 12_Warpings, 5.3ms
image 6/250 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Cap

In [16]:
from ultralytics import YOLO

model = YOLO("runs/detect/train_final/weights/best.pt")

In [17]:
model.predict(
    source="data/test/images",  # wherever your test images are
    save=True,
    name="test",
    save_txt=True,     # saves YOLO-format predictions (class x y w h conf) per image
    save_conf=True
)


image 1/300 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\00624767-right_77photo_2024-10-04T11_08_33-787900.jpg: 416x640 1 7_Spaghetti, 49.5ms
image 2/300 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\009529f9-right_photo_182_2024-10-22_17-27-24.jpg: 416x640 7 13_Delaminations, 57.0ms
image 3/300 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\00a8a190-left_1750photo_2024-08-27T17_19_02-958622.jpg: 416x640 4 4_Foreign_Bodys, 43.0ms
image 4/300 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\027bae6a-left_photo_245_2024-11-21_18-00-35.jpg: 416x640 4 4_Foreign_Bodys, 18.1ms
image 5/300 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\042994e7-left_right_181photo_2024-10-02T14_43_58-554388.jpg: 416x640 1 6_Stringing, 18.8ms
image 6/300 c:\Users\User\Documents\GitHub\Re

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '01_Good', 1: '4_Foreign_Body', 2: '6_Stringing', 3: '7_Spaghetti', 4: '9_Poor_Bridging', 5: '10_Overhang_Sag', 6: '11_Shifted_Layer', 7: '12_Warping', 8: '13_Delamination', 9: '14_Unclear', 10: '15_Not_Printing'}
 obb: None
 orig_img: array([[[ 28,  24,  23],
         [ 28,  24,  23],
         [ 28,  24,  23],
         ...,
         [ 55,  53,  52],
         [ 55,  53,  52],
         [ 55,  53,  52]],
 
        [[ 29,  25,  24],
         [ 28,  24,  23],
         [ 28,  24,  23],
         ...,
         [ 54,  52,  51],
         [ 54,  52,  51],
         [ 54,  52,  51]],
 
        [[ 29,  25,  24],
         [ 29,  25,  24],
         [ 28,  24,  23],
         ...,
         [ 54,  52,  51],
         [ 54,  52,  51],
         [ 53,  51,  50]],
 
        ...,
 
        [[199, 197, 187],
         [204, 202, 192],
         [201, 199, 189],
 

In [18]:
model.val(data="data/data.yaml",
          split="test",
          device=0,
          name="test_metrics"
)

print("✅ Test evaluation complete.")

Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 1359.7251.9 MB/s, size: 290.4 KB)
val: Scanning C:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\labels... 300 images, 8 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 1.0Kit/s 0.3s<0.2s
val: New cache created: C:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 6.6it/s 2.9s0.1s
                   all        300        700      0.772      0.673      0.711      0.416
        4_Foreign_Body         37        104      0.786      0.894      0.866      0.485
           6_Stringing         42         84      0.806      0.667      0.716      0.429
           7_Spaghetti         43         79      0.703      0.646      0.682      0.456
       9_Po